<a href="https://colab.research.google.com/github/AshinDevUA/GNN/blob/main/GNN_code_grocery_sales_forecasting_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import pandas as pd
import numpy as np

In [44]:
# Creating a directory called 'dataset'
!mkdir 'dataset'

In [45]:
# Verify the directory was created
!ls -l

total 468656
drwxr-xr-x 2 root root      4096 Nov 26 17:41 dataset
-rw-r--r-- 1 root root      1898 Dec 11  2019 holidays_events.csv.7z
-rw-r--r-- 1 root root     14315 Dec 11  2019 items.csv.7z
-rw-r--r-- 1 root root      3762 Dec 11  2019 oil.csv.7z
-rw-r--r-- 1 root root    666528 Dec 11  2019 sample_submission.csv.7z
-rw-r--r-- 1 root root       648 Dec 11  2019 stores.csv.7z
-rw-r--r-- 1 root root   4885065 Dec 11  2019 test.csv.7z
-rw-r--r-- 1 root root 474092593 Dec 11  2019 train.csv.7z
-rw-r--r-- 1 root root    219499 Dec 11  2019 transactions.csv.7z


In [46]:
%cd 'dataset'
!unzip '/content/drive/MyDrive/project/favorita-grocery-sales-forecasting'

/content/dataset/dataset
Archive:  /content/drive/MyDrive/project/favorita-grocery-sales-forecasting.zip
  inflating: holidays_events.csv.7z  
  inflating: items.csv.7z            
  inflating: oil.csv.7z              
  inflating: sample_submission.csv.7z  
  inflating: stores.csv.7z           
  inflating: test.csv.7z             
  inflating: train.csv.7z            
  inflating: transactions.csv.7z     


In [47]:
import pandas as pd
import numpy as np
import os

# Paths to the .csv files
datasets_paths = {
    'holidays_events': '/content/drive/MyDrive/project/holidays_events.csv',
    'items': '/content/drive/MyDrive/project/items.csv',
    'oil': '/content/drive/MyDrive/project/oil.csv',
    'sample_submission': '/content/drive/MyDrive/project/sample_submission.csv',
    'stores': '/content/drive/MyDrive/project/stores.csv',
    'train': '/content/drive/MyDrive/project/train.csv',
    'test': '/content/drive/MyDrive/project/test.csv',
    'transactions': '/content/drive/MyDrive/project/transactions.csv'
}

In [8]:
# Read the datasets directly using pd.read_csv
holidays_events = pd.read_csv(datasets_paths['holidays_events'], encoding='ISO-8859-1')
items = pd.read_csv(datasets_paths['items'], encoding='ISO-8859-1')
oil = pd.read_csv(datasets_paths['oil'], encoding='ISO-8859-1')
sample_submission = pd.read_csv(datasets_paths['sample_submission'], encoding='ISO-8859-1')
stores = pd.read_csv(datasets_paths['stores'], encoding='ISO-8859-1')

# Specify dtype to handle mixed types
train = pd.read_csv(datasets_paths['train'], encoding='ISO-8859-1', dtype={'onpromotion': str})
test = pd.read_csv(datasets_paths['test'], encoding='ISO-8859-1')
transactions = pd.read_csv(datasets_paths['transactions'], encoding='ISO-8859-1')


In [9]:
# Convert 'date' columns to datetime format
train['date'] = pd.to_datetime(train['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
transactions['date'] = pd.to_datetime(transactions['date'])

In [10]:
# Use ffill() to fill NaN values
oil['dcoilwtico'] = oil['dcoilwtico'].ffill()

In [11]:
# Define date ranges for filtering
train_start_date = '2017-03-01'
train_end_date = '2017-06-21'
validation_start_date = '2017-06-28'
validation_end_date = '2017-07-13'
test_start_date = '2017-07-19'
test_end_date = '2017-07-23'

# Filter train dataset by date ranges
train_filtered = train[(train['date'] >= train_start_date) & (train['date'] <= train_end_date)]
validation_filtered = train[(train['date'] >= validation_start_date) & (train['date'] <= validation_end_date)]
test_filtered = train[(train['date'] >= test_start_date) & (train['date'] <= test_end_date)]

In [12]:
# Merge datasets with stores, items, oil, holidays, and transactions based on date
def merge_datasets(data):
    # Merge with stores and items
    data = data.merge(stores, on='store_nbr', how='left')
    data = data.merge(items, on='item_nbr', how='left')
    data = data.merge(oil, on='date', how='left')
    data = data.merge(holidays_events, on='date', how='left')
    data = data.merge(transactions, on=['date', 'store_nbr'], how='left')
    return data

# Apply merging
train_merged = merge_datasets(train_filtered)
validation_merged = merge_datasets(validation_filtered)
test_merged = merge_datasets(test_filtered)

# Print the shapes of the merged datasets
print(f"Merged training dataset shape: {train_merged.shape}")
print(f"Merged validation dataset shape: {validation_merged.shape}")
print(f"Merged test dataset shape: {test_merged.shape}")

Merged training dataset shape: (12051403, 20)
Merged validation dataset shape: (1807983, 20)
Merged test dataset shape: (523365, 20)


In [48]:
# Fill NaN values with mode for each_dataset and apply infer_objects
train_data = train_merged.fillna(train_merged.mode().iloc[0]).infer_objects()
validation_data = validation_merged.fillna(validation_merged.mode().iloc[0]).infer_objects()
test_data = test_merged.fillna(test_merged.mode().iloc[0]).infer_objects()

# Print the shapes of the reduced_datasets
print(f"training dataset shape  : {train_data.shape}")
print(f"validation dataset shape: {validation_data.shape}")
print(f"test dataset shape      : {test_data.shape}")

<ipython-input-48-9efc3751797e>:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

<ipython-input-48-9efc3751797e>:3: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



training dataset shape  : (12051403, 20)
validation dataset shape: (1807983, 20)
test dataset shape      : (523365, 20)


<ipython-input-48-9efc3751797e>:4: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [14]:
train_data.shape

(12051403, 20)

In [17]:
test_data.head(5)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type_x,cluster,family,class,perishable,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
0,122566434,2017-07-19,1,99197,2.0,False,Quito,Pichincha,D,13,GROCERY I,1067,0,47.1,Holiday,Local,Cayambe,Cantonizacion de Cayambe,False,1797
1,122566435,2017-07-19,1,103520,1.0,False,Quito,Pichincha,D,13,GROCERY I,1028,0,47.1,Holiday,Local,Cayambe,Cantonizacion de Cayambe,False,1797
2,122566436,2017-07-19,1,103665,3.0,False,Quito,Pichincha,D,13,BREAD/BAKERY,2712,1,47.1,Holiday,Local,Cayambe,Cantonizacion de Cayambe,False,1797
3,122566437,2017-07-19,1,105574,4.0,False,Quito,Pichincha,D,13,GROCERY I,1045,0,47.1,Holiday,Local,Cayambe,Cantonizacion de Cayambe,False,1797
4,122566438,2017-07-19,1,105575,12.0,False,Quito,Pichincha,D,13,GROCERY I,1045,0,47.1,Holiday,Local,Cayambe,Cantonizacion de Cayambe,False,1797


### ***EDA***:

In [18]:
import pandas as pd
import plotly.express as px

# Function to plot unit sales over time for a given dataset
def plot_sales_over_time(data, title):
    sales_time_series = data.groupby('date')['unit_sales'].sum().reset_index()
    fig = px.line(sales_time_series, x='date', y='unit_sales', title=title)
    fig.update_layout(xaxis_title='Date', yaxis_title='Total Unit Sales', xaxis_rangeslider_visible=True)
    fig.show()

# Plot each dataset
plot_sales_over_time(train_data, 'Unit Sales Over Time (Train Data)')
plot_sales_over_time(test_data, 'Unit Sales Over Time (Test Data)')
plot_sales_over_time(validation_data, 'Unit Sales Over Time (Validation Data)')

In [19]:
# Function to plot average unit sales based on promotion status for a given dataset
def plot_promo_sales(data, title, color):
    promo_sales = data.groupby('onpromotion')['unit_sales'].mean().reset_index()
    fig = px.bar(promo_sales, x='onpromotion', y='unit_sales', title=title, color_discrete_sequence=[color])
    fig.update_layout(xaxis_title='On Promotion', yaxis_title='Average Unit Sales')
    fig.show()

# Plot each dataset with different colors
plot_promo_sales(train_data, 'Average Unit Sales: Promotion vs No Promotion (Train Data)', 'blue')
plot_promo_sales(test_data, 'Average Unit Sales: Promotion vs No Promotion (Test Data)', 'green')
plot_promo_sales(validation_data, 'Average Unit Sales: Promotion vs No Promotion (Validation Data)', 'red')

In [20]:
# Function to plot unit sales by family in a sunburst chart for a given dataset
def plot_sunburst_sales(data, title, color_sequence):
    sales_by_family = data.groupby(['family'])['unit_sales'].sum().reset_index()
    fig = px.sunburst(sales_by_family, path=['family'], values='unit_sales', title=title, color_discrete_sequence=color_sequence)
    fig.show()

# Plot each dataset with different color sequences
plot_sunburst_sales(train_data, 'Unit Sales by Item Family (Train Data)', px.colors.qualitative.Pastel)
plot_sunburst_sales(test_data, 'Unit Sales by Item Family (Test Data)', px.colors.qualitative.Vivid)
plot_sunburst_sales(validation_data, 'Unit Sales by Item Family (Validation Data)', px.colors.qualitative.Prism)

In [21]:
import pandas as pd
import plotly.express as px

# Function to plot unit sales by city and family in a sunburst chart for a given dataset
def plot_sunburst_sales(data, title, color_sequence):
    sales_by_city_family = data.groupby(['city', 'family'])['unit_sales'].sum().reset_index()
    fig = px.sunburst(sales_by_city_family, path=['city', 'family'], values='unit_sales', title=title, color_discrete_sequence=color_sequence)
    fig.show()

# Plot each dataset with different color sequences
plot_sunburst_sales(train_data, 'Unit Sales by City and Item Family (Train Data)', px.colors.qualitative.Pastel)
plot_sunburst_sales(test_data, 'Unit Sales by City and Item Family (Test Data)', px.colors.qualitative.Vivid)
plot_sunburst_sales(validation_data, 'Unit Sales by City and Item Family (Validation Data)', px.colors.qualitative.Prism)

In [22]:
# Function to plot top 10 cities by total unit sales in a horizontal bar chart
def plot_top_cities_sales(data, title, color):
    sales_by_city = data.groupby('city')['unit_sales'].sum().sort_values(ascending=False).head(10).reset_index()
    fig = px.bar(sales_by_city, x='unit_sales', y='city', title=title, orientation='h', color_discrete_sequence=[color])
    fig.update_layout(xaxis_title='Total Unit Sales', yaxis_title='City')
    fig.show()

# Plot each dataset with different colors
plot_top_cities_sales(train_data, 'Top 10 Cities by Total Unit Sales (Train Data)', 'blue')
plot_top_cities_sales(test_data, 'Top 10 Cities by Total Unit Sales (Test Data)', 'green')
plot_top_cities_sales(validation_data, 'Top 10 Cities by Total Unit Sales (Validation Data)', 'red')

In [23]:
train_data.head(2)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type_x,cluster,family,class,perishable,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
0,107758056,2017-03-01,1,105574,10.0,False,Quito,Pichincha,D,13,GROCERY I,1045,0,53.82,Holiday,National,Ecuador,Fundacion de Cuenca,False,1873
1,107758057,2017-03-01,1,105575,18.0,False,Quito,Pichincha,D,13,GROCERY I,1045,0,53.82,Holiday,National,Ecuador,Fundacion de Cuenca,False,1873


In [24]:
# Function to plot unit sales by city and store type in a stacked bar chart for a given dataset
def plot_sales_by_city_type(data, title, color_sequence):
    sales_by_city_type = data.groupby(['city', 'type_x'])['unit_sales'].sum().reset_index()
    fig = px.bar(sales_by_city_type, x='city', y='unit_sales', color='type_x',
                 title=title, barmode='stack', color_discrete_sequence=color_sequence)
    fig.update_layout(xaxis_title='City', yaxis_title='Total Unit Sales')
    fig.show()

# Plot each dataset with different color sequences
plot_sales_by_city_type(train_data, 'Total Unit Sales by City and Store Type (Train Data)', px.colors.qualitative.Pastel)
plot_sales_by_city_type(test_data, 'Total Unit Sales by City and Store Type (Test Data)', px.colors.qualitative.Vivid)
plot_sales_by_city_type(validation_data, 'Total Unit Sales by City and Store Type (Validation Data)', px.colors.qualitative.Prism)

In [26]:
train_data.dtypes

,0
id,int64
date,datetime64[ns]
store_nbr,int64
item_nbr,int64
unit_sales,float64
onpromotion,object
city,object
state,object
type_x,object
cluster,int64


In [27]:
print(train_data.isnull().sum())

id              0
date            0
store_nbr       0
item_nbr        0
unit_sales      0
onpromotion     0
city            0
state           0
type_x          0
cluster         0
family          0
class           0
perishable      0
dcoilwtico      0
type_y          0
locale          0
locale_name     0
description     0
transferred     0
transactions    0
dtype: int64


In [ ]:
# # Drop specified columns from train_data
# columns_to_drop = ['dcoilwtico', 'type_y', 'locale', 'locale_name', 'description', 'transferred']
# train_data = train_data.drop(columns=columns_to_drop, errors='ignore')

In [25]:
#train_data.shape

(12051403, 20)

In [27]:
#train_data.tail(2)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type_x,cluster,family,class,perishable,dcoilwtico,type_y,locale,locale_name,description,transferred,transactions
12051401,119707614,2017-06-21,54,2103250,2.0,True,El Carmen,Manabi,C,3,BEAUTY,4254,0,42.48,Holiday,National,Ecuador,Fundacion de Cuenca,False,658
12051402,119707615,2017-06-21,54,2106464,1.0,True,El Carmen,Manabi,C,3,BEVERAGES,1148,0,42.48,Holiday,National,Ecuador,Fundacion de Cuenca,False,658


In [28]:
#train_data.shape

(12051403, 20)

In [28]:
# Assuming 'train_data' is already defined as in the provided code.
train_data['date'] = pd.to_datetime(train_data['date'])
train_data = train_data.set_index('date')

test_data['date'] = pd.to_datetime(test_data['date'])
test_data = test_data.set_index('date')

validation_data['date'] = pd.to_datetime(validation_data['date'])
validation_data = validation_data.set_index('date')

In [29]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12051403 entries, 2017-03-01 to 2017-06-21
Data columns (total 19 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            int64  
 1   store_nbr     int64  
 2   item_nbr      int64  
 3   unit_sales    float64
 4   onpromotion   object 
 5   city          object 
 6   state         object 
 7   type_x        object 
 8   cluster       int64  
 9   family        object 
 10  class         int64  
 11  perishable    int64  
 12  dcoilwtico    float64
 13  type_y        object 
 14  locale        object 
 15  locale_name   object 
 16  description   object 
 17  transferred   bool   
 18  transactions  int64  
dtypes: bool(1), float64(2), int64(7), object(9)
memory usage: 1.7+ GB


In [30]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Function to apply label encoding
def label_encode_columns(df):
    # Identify columns with object and bool data types
    object_columns = df.select_dtypes(include=['object']).columns
    bool_columns = df.select_dtypes(include=['bool']).columns

    # Combine the columns to be encoded
    columns_to_encode = object_columns.append(bool_columns)

    # Apply LabelEncoder to each specified column
    for column in columns_to_encode:
        df[column] = label_encoder.fit_transform(df[column])

    return df

# Apply label encoding to each dataset
train_data = label_encode_columns(train_data)
validation_data = label_encode_columns(validation_data)
test_data = label_encode_columns(test_data)

In [31]:
trn_data = train_data['unit_sales']
val_data = validation_data['unit_sales']
tst_data = test_data['unit_sales']

In [ ]:
# # Assuming 'train_data' is already defined as in the provided code.
# import pandas as pd

# # Assuming 'train_data' is already defined and has a 'unit_sales' column and a datetime index
# def create_sliding_window_dataset(data, window_size):
#     X = []
#     y = []
#     for i in range(len(data) - window_size):
#         X.append(data[i:i + window_size])
#         y.append(data[i + window_size])  # Target is the next value after the window

#     return np.array(X), np.array(y)

# window_size = 5
# X_trn, y_trn = create_sliding_window_dataset(trn_data, window_size)
# # X_val, y_val = create_sliding_window_dataset(val_data, window_size)
# # X_tst, y_tst = create_sliding_window_dataset(tst_data, window_size)
# print("X_trn shape:", X_trn.shape)
# print("y_trn shape:", y_trn.shape)
# # print("X_val shape:", X_val.shape)
# # print("y_val shape:", y_val.shape)
# # print("X_tst shape:", X_tst.shape)
# # print("y_tst shape:", y_tst.shape)

In [33]:
train_data.columns

Index(['id', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'city',
       'state', 'type_x', 'cluster', 'family', 'class', 'perishable',
       'dcoilwtico', 'type_y', 'locale', 'locale_name', 'description',
       'transferred', 'transactions'],
      dtype='object')

In [34]:
train_data.dtypes

,0
id,int64
store_nbr,int64
item_nbr,int64
unit_sales,float64
onpromotion,int64
city,int64
state,int64
type_x,int64
cluster,int64
family,int64


In [35]:
# Train dataset
X_train = train_data.drop(columns=['id', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'city',
       'state', 'type_x', 'cluster', 'family', 'class', 'perishable',
       'dcoilwtico', 'type_y', 'locale', 'locale_name', 'description',
       'transferred'])
y_train = train_data['unit_sales']

# Test dataset
X_test = test_data.drop(columns=['id', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'city',
       'state', 'type_x', 'cluster', 'family', 'class', 'perishable',
       'dcoilwtico', 'type_y', 'locale', 'locale_name', 'description',
       'transferred'])
y_test = test_data['unit_sales']

# Validation dataset
X_val = validation_data.drop(columns=['id', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'city',
       'state', 'type_x', 'cluster', 'family', 'class', 'perishable',
       'dcoilwtico', 'type_y', 'locale', 'locale_name', 'description',
       'transferred'])
y_val = validation_data['unit_sales']

In [36]:
X_train_re = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_re = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test_re = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

In [37]:
from keras.utils import to_categorical
y_train1=to_categorical(y_train)
y_val1=to_categorical(y_val)
y_test1=to_categorical(y_test)

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
import numpy as np


# Define the GNN model
def create_GNN_model(input_shape):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(filters=32, kernel_size=1, activation='relu'), # Changed kernel_size to 1
        layers.MaxPooling1D(pool_size=1), # Changed pool_size to 1 to accommodate smaller kernel
        layers.Conv1D(filters=64, kernel_size=1, activation='relu'), # Changed kernel_size to 1
        layers.MaxPooling1D(pool_size=1), # Changed pool_size to 1 to accommodate smaller kernel
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1)  # Output layer for regression
    ])
    return model

# Create and compile the model
model = create_GNN_model(input_shape=(X_train_re.shape[1], 1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train_re, y_train, epochs=10, batch_size=128, validation_data=(X_val_re, y_val))

Epoch 1/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 153s 2ms/step - loss: 653.3018 - val_loss: 503.7280
Epoch 2/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 144s 2ms/step - loss: 730.9257 - val_loss: 503.5594
Epoch 3/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 144s 2ms/step - loss: 657.5800 - val_loss: 503.4933
Epoch 4/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 144s 2ms/step - loss: 653.3094 - val_loss: 503.4554
Epoch 5/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 145s 2ms/step - loss: 601.8813 - val_loss: 503.4941
Epoch 6/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 145s 2ms/step - loss: 662.5974 - val_loss: 503.5280
Epoch 7/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 144s 2ms/step - loss: 709.5162 - val_loss: 503.5435
Epoch 8/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 144s 2ms/step - loss: 657.5327 - val_loss: 503.4906
Epoch 9/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 144s 2ms/step - loss: 613.8668 - val_loss: 503.4358
Epoch 10/10
94152/94152 ━━━━━━━━━━━━━━━━━━━━ 145s 2ms/step - loss: 589.9284 - val_loss: 503.5905


In [41]:
# Make predictions
y_pred = model.predict(X_test_re)

# Calculate RMSLE
def rmsle(y_true, y_pred):
    # Clip y_true and y_pred to avoid negative or zero values
    y_true_clipped = np.clip(y_true, a_min=1e-6, a_max=None)
    y_pred_clipped = np.clip(y_pred, a_min=1e-6, a_max=None)
    return np.sqrt(mean_squared_error(np.log1p(y_true_clipped), np.log1p(y_pred_clipped)))

# Calculate RMSWLE
def rmswle(y_true, y_pred, weights):
    weighted_squared_errors = weights * (np.log1p(y_true) - np.log1p(y_pred))**2
    return np.sqrt(np.mean(weighted_squared_errors))

# Calculate NWRMSLE (assuming equal weights for simplicity)
def nwrmsle(y_true, y_pred):
    return rmsle(y_true, y_pred)

# Calculate RMALE (assuming weights are available)
def rmale(y_true, y_pred, weights):
    absolute_errors = np.abs(y_true - y_pred)
    weighted_absolute_errors = weights * absolute_errors
    return np.mean(weighted_absolute_errors)

rmsle_score = rmsle(y_test, y_pred)
#rmswle_score = rmswle(y_test, y_pred, weights)
nwrmsle_score = nwrmsle(y_test, y_pred)
#rmale_score = rmale(y_test, y_pred, weights)


print(f"RMSLE: {rmsle_score}")
#print(f"RMSWLE: {rmswle_score}")
print(f"NWRMSLE: {nwrmsle_score}")
#print(f"RMALE: {rmale_score}")

16356/16356 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step
RMSLE: 0.9880118808470838
NWRMSLE: 0.9880118808470838
